# Projeto 1 - Ciência dos Dados

Nome: Breno Brolacci Schiavon

Nome: João Manoel Pasqua Filho

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [139]:
#importando bibliotecas:

%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [140]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\João Manoel\Documents\cdados\projeto1cdados (2)\projeto1cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [141]:
filename = 'BigMac1.xlsx'

In [142]:
#Abrindo arquivo na parte de treinamento:

train = pd.read_excel(filename)
train.head()

,Treinamento,Relevancia
0,@tiagogreis o frango tá o preço do big mac,0
1,somente um combo de duplo big mac agora pra me...,1
2,queria tanto um big mac agr,1
3,mãe esqueceu de trazer o meu almoço então feli...,1
4,quando eu e o boy tá perto do mcdonald's ele j...,1


In [143]:
#Abrindo arquivo na parte de teste:

test = pd.read_excel(filename, sheet_name = 'Teste')
test.head()

,Teste,Relevância
0,"@ancapsu tenho medo, o brasil vai conseguir de...",0.0
1,@tabataamaralsp big mac em promoção hoje no 7 ...,1.0
2,"almocei frango, alface, cebola, abobora caboti...",1.0
3,qria um big mac agr,1.0
4,"@alcrsk @objetoscomcara mano, acho que ela é u...",0.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

O produto escolhido foi o lanche "Big mac" do restaurante fast-food Mc Donalds. Para classificar a relevância de um tweet, decidimos optar por selecionar como relevante tudo que qualificava o lanche, sendo uma opinião negativa ou positiva.


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [144]:
#funcao para remover a pontuacao dos tweets:

def sem_pontuacao(x):
    
    pontuacao = '[,"!-.:?;]'
    padrao = re.compile(pontuacao)
    texto = re.sub(padrao, '', x)
    return texto


In [145]:
#listas para adicionar as palavras dos tweets:

palavras_treinamento_lista = []
palavras_relevantes_treinamento_lista = []
palavras_irrelevantes_treinamento_lista = []

#loop (tira a pontuacao, muda tudo para letra minuscula e adiciona em uma lista)

for i in range(0,len(train)):
    tweet = train["Treinamento"].iloc[i]
    relevancia = train["Relevancia"].iloc[i]
    train["Treinamento"].iloc[i] = sem_pontuacao(tweet.lower())
    for palavra in tweet.split():
        palavras_treinamento_lista.append(sem_pontuacao(palavra))
        
        if relevancia == 1:
            palavras_relevantes_treinamento_lista.append(sem_pontuacao(palavra))
        
        else:
            palavras_irrelevantes_treinamento_lista.append(sem_pontuacao(palavra))


In [146]:
#transforma a lista em uma Series para depois fazer a contagem dos valores:

palavras_treinamento = pd.Series(palavras_treinamento_lista)
tabela_palavras = palavras_treinamento.value_counts(True)
tabela_palavras.head()

big    0.062921
mac    0.059975
um     0.033880
e      0.027146
de     0.023148
dtype: float64

In [147]:
#transforma a lista em uma Series para depois fazer a contagem dos valores:

palavras_relevantes_treinamento = pd.Series(palavras_relevantes_treinamento_lista)
tabela_palavras_relevantes = palavras_relevantes_treinamento.value_counts(True)
tabela_palavras_relevantes.head()

big    0.068531
mac    0.066073
um     0.040565
e      0.025814
de     0.022127
dtype: float64

In [148]:
#transforma a lista em uma Series para depois fazer a contagem dos valores:

palavras_irrelevantes_treinamento = pd.Series(palavras_irrelevantes_treinamento_lista)
tabela_palavras_irrelevantes = palavras_irrelevantes_treinamento.value_counts(True)
tabela_palavras_irrelevantes.head()

big    0.050734
mac    0.046729
e      0.030040
o      0.027370
de     0.025367
dtype: float64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [149]:
#Probabilidade de ser Relevante:

ProbR = len(palavras_relevantes_treinamento)/len(palavras_treinamento)

#Probabilidade de ser Irrelevante:

ProbI = len(palavras_irrelevantes_treinamento)/len(palavras_treinamento)
            
print(f'P(R)={ProbR} \nP(I)={ProbI}')


P(R)=0.6847643097643098 
P(I)=0.31523569023569026


In [154]:
frase_limpa = ""

contagem_relevantes = tabela_palavras_relevantes.shape[0]
contagem_irrelevantes = tabela_palavras_irrelevantes.shape[0]

#Adicionando uma nova coluna para o Dataframe do teste:
test['Relevancia_bayes']=0

total_palavras = len(palavras_treinamento_lista)

ocorrencias = 0

#Percorrendo palavra por palavra dos tweets para calcular a probabilidade do tweet ser relevante ou nao,
# tudo isso baseado na frequencia de aparicao das palavras na base de treinamento:

for i in range(0,len(test)):
    probTweetDadoR = 1
    probTweetDadoI = 1
   
    frase = test["Teste"].iloc[i]
    frase_limpa = sem_pontuacao(frase.lower())
    lista_palavras = frase_limpa.split()
    
    for palavra in lista_palavras:
        ocorrencia_relevante = 0
        ocorrencia_irrelevante = 0
        
        if palavra in tabela_palavras_relevantes:
            ocorrencia_relevante = tabela_palavras_relevantes[palavra]
            
        if palavra in tabela_palavras_irrelevantes:
            ocorrencia_irrelevante = tabela_palavras_irrelevantes[palavra]
            
        probTweetDadoR *= (ocorrencia_relevante + 1) / (total_palavras + contagem_relevantes)
        probTweetDadoI *= ( ocorrencia_irrelevante + 1) / (total_palavras + contagem_irrelevantes)

    pRtweet = probTweetDadoR * ProbR

    pItweet = probTweetDadoI * ProbI
    
    if pRtweet > pItweet:
        test['Relevancia_bayes'][i] = 1
          
test.head()


C:\Apps\Anaconda\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Teste,Relevância,Relevancia_bayes
0,"@ancapsu tenho medo, o brasil vai conseguir de...",0.0,0
1,@tabataamaralsp big mac em promoção hoje no 7 ...,1.0,0
2,"almocei frango, alface, cebola, abobora caboti...",1.0,0
3,qria um big mac agr,1.0,1
4,"@alcrsk @objetoscomcara mano, acho que ela é u...",0.0,1


In [151]:
#Tabela que mostra o percentual de acertos para relacionando a tabela de relevancia classificada 
# manual com a tabela do classificador.
pd.crosstab(test['Relevância'], test['Relevancia_bayes'], normalize='all').round(4) * 100

Relevancia_bayes,0,1
Relevância,,
0.0,18.09,15.58
1.0,35.68,30.65


In [152]:
#Calculando acuracia atraves de um loop que faz a contagem dos valores que coincidem e divide pelo numero de termos.

acuracia = 0
termos = 0
for i in range(0, len(test)):
    if test['Relevancia_bayes'][i] == test['Relevância'][i]:
        acuracia += 1
    termos += 1

acuracia = (acuracia/termos) * 100

print('A acurácia do classificador é igual a {} %'.format(acuracia))

A acurácia do classificador é igual a 48.5 %


___
### Concluindo

A partir da acurácia de nosso classificador, pode-se perceber que há muitas coisas que podem ser feitas para um aumento desse percentual (48.5%), como por exemplo a adaptação de abreviações de palavras, a categorização de significado de emojis com o intuito de entender o sentimento de uma pessoa a partir do seu uso. Além disso, conseguimos perceber que algumas figuras de linguagem utilizadas, como o sarcasmo por exemplo, interferem no resultado da eficiência do classificador de modo que o computador não consegue entender o duplo sentido de uma frase e acaba o classificando de maneira errônea. Com isso, acreditamos que mesmo após obtermos um baixo percentual de acurácia, com a implementação de melhorias citadas anteriormente e outras não citadas, o desempenho do classificador pode aumentar muito, sendo assim um projeto muito promissor.



Com o desenolvimento do projeto, conseguimos aprender muito sobre o classificador de Naive-Bayes e concluímos que é possível a utilização dessa ferramenta em diversas ocasiões, como por exemplo uma ferramenta de pesquisa de uma biblioteca, que a partir de uma citação, descobre o livro a qual pertence.

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**